# 10 Minutes to cuDF and CuPy

This notebook provides introductory examples of how you can use cuDF and CuPy together to take advantage of CuPy array functionality (such as advanced linear algebra operations).

In [ ]:
import time

from numba import cuda

import cupy as cp
import cudf


### Converting a cuDF DataFrame to a CuPy Array

If we want to convert a cuDF DataFrame to a CuPy ndarray, There are multiple ways to do it:

1. The best way is to use the [dlpack](https://github.com/dmlc/dlpack) interface.

2. We can also convert via the [CUDA array interface](https://numba.pydata.org/numba-doc/dev/cuda/cuda_array_interface.html) by using cuDF's `as_gpu_matrix` and CuPy's `asarray` functionality. Because CuPy arrays have a single dtype, each column in our DataFrame must have the same dtype, regardless of which method we use.

In [ ]:
nelem = 10000
df = cudf.DataFrame({'a':range(nelem),
                     'b':range(500, nelem + 500),
                     'c':range(1000, nelem + 1000)}
                   )

%time arr_cupy = cp.fromDlpack(df.to_dlpack())
arr_cupy

In [ ]:
cp.asarray(df.as_gpu_matrix())

### Converting a cuDF Series to a CuPy Array

There are multiple ways to convert a cuDF Series to a CuPy array:

1. **Easiest & Preferred**: You can convert a cuDF Series to a CuPy by passing the Series to `cupy.asarray` as cuDF Series exposes [`__cuda_array_interface__`](https://docs-cupy.chainer.org/en/stable/reference/interoperability.html)
2. By passing the underlying Numba DeviceNDArray to `cupy.asarray`.
3. We can also leverage the dlpack interface. `to_dlpack()`


In [ ]:
col = 'a'

%time cola_cupy = cp.asarray(df[col])
%time cola_cupy = cp.asarray(df[col].data)
%time cola_cupy = cp.fromDlpack(df[col].to_dlpack())

type(cola_cupy)

From here, we can proceed with normal CuPy workflows, such as reshaping the array, getting the diagonal, or calculating the norm.

In [ ]:
reshaped_arr = cola_cupy.reshape(50, 200)
reshaped_arr

In [ ]:
reshaped_arr.diagonal()

In [ ]:
cp.linalg.norm(reshaped_arr)

### Converting a CuPy Array to a cuDF DataFrame

We can also convert a CuPy ndarray to a cuDF DataFrame. As above, we can use the either the dlpack interface or CUDA array interface with cuDF's `from_gpu_matrix`. Either way, we'll need to make sure that our CuPy array is Fortran contiguous in memory (if it's not already). We can either transpose the array or simply coerce it to be Fortran contiguous beforehand.

We can check whether our array is Fortran contiguous by using `cupy.isfortran` or looking at the [flags](https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.ndarray.html#cupy.ndarray.flags) of the array.

In [ ]:
cp.isfortran(reshaped_arr)

In this case, we'll need to convert it before going to a cuDF DataFrame. In the next two cells, we create the DataFrame by leveraging dlpack and the CUDA array interface, respectively.

In [ ]:
reshaped_arr = cp.asfortranarray(reshaped_arr)
reshaped_df = cudf.from_dlpack(reshaped_arr.toDlpack())
reshaped_df.head()

In [ ]:
reshaped_df = cudf.DataFrame.from_gpu_matrix(reshaped_arr)
reshaped_df.head()

### Converting a CuPy Array to a cuDF Series

To convert an array to a Series, we can directly pass the array to the constructor. We just need to make sure that the array is stored in contiguous memory. If it's not, we need to create a contiguous array with `ascontiguousarray`. We could also use `asfortranarray`, but it won't matter in the case of this one-dimensional array.

In [ ]:
diag_data = cp.ascontiguousarray(reshaped_arr.diagonal())
cudf.Series(diag_data).head()

### Interweaving CuDF and CuPy for Smooth PyData Workflows

RAPIDS libraries and the entire GPU PyData ecosystem are developing quickly, but sometimes a one library may not have the functionality you need. One example of this might be taking the row-wise sum (or mean) of a Pandas DataFrame. cuDF's support for row-wise operations isn't mature, so you'd need to either transpose the DataFrame or write a UDF and explicitly calculate the sum across each row. Transposing could lead to hundreds of thousands of columns (which cuDF wouldn't perform well with) depending on your data's shape, and writing a UDF can be time intensive.

By leveraging the interoperability of the GPU PyData ecosystem, this operation becomes very easy. Let's take the row-wise sum of our previously reshaped cuDF DataFrame.

In [ ]:
reshaped_df.head()

We can just transform it into a CuPy array via dlpack and use the `axis` argument of `sum`.

In [ ]:
new_arr = cp.fromDlpack(reshaped_df.to_dlpack())
new_arr.sum(axis=1)

With just that single line, we're able to seamlessly move between data structures in this ecosystem, giving us enormous flexibility without sacrificing speed.

### Converting a cuDF DataFrame to a CuPy Sparse Matrix

We can also convert a DataFrame or Series to a CuPy sparse matrix. We might want to do this if downstream processes expect CuPy sparse matrices as an input.

The sparse matrix data structure is defined by three dense arrays, which we could create manually from an existing cuDF DataFrame or Series. Luckily, we don't need to do that. We can simply leverage dlpack again. We'll define a small helper function for cleanliness.

In [ ]:
def cudf_to_cupy_sparse_matrix(data, sparseformat='column'):
    """Converts a cuDF object to a CuPy Sparse Column matrix.
    """
    if sparseformat not in ('row', 'column',):
        raise ValueError("Let's focus on column and row formats for now.")
    
    _sparse_constructor = cp.sparse.csc_matrix
    if sparseformat == 'row':
        _sparse_constructor = cp.sparse.csr_matrix

    return _sparse_constructor(cp.fromDlpack(data.to_dlpack()))

We can define a sparsely populated DataFrame to illustrate this conversion to either sparse matrix format.

In [ ]:
df = cudf.DataFrame()
nelem = 10000
nonzero = 1000
for i in range(20):
    arr = cp.random.normal(5, 5, nelem)
    arr[cp.random.choice(arr.shape[0], nelem-nonzero, replace=False)] = 0
    df['a' + str(i)] = cp.ascontiguousarray(arr)

In [ ]:
df.head()

In [ ]:
sparse_data = cudf_to_cupy_sparse_matrix(df)
sparse_data

From here, we could continue our workflow with a CuPy sparse matrix.

For a full list of the functionality built into these libraries, we encourage you to check out the API docs for [cuDF](https://docs.rapids.ai/api/cudf/nightly/) and [CuPy](https://docs-cupy.chainer.org/en/stable/index.html).